In [1]:
from IPython.display import Markdown, display

In [2]:
def printmd(string):
    display(Markdown(string))

### Tilt-series Alignment (IMOD)

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import yaml

In [ ]:
stats_yaml_file = f"{proj_name}_imod_align_stats.yaml"

with open(stats_yaml_file, "r") as f:
    stats = yaml.load(f.read(), Loader=yaml.FullLoader)
    
stats_df = pd.DataFrame.from_dict(stats).drop(columns=["index"])
stats_df["Tilt series"] = stats_df["Tilt series"].astype(int, copy=False)

In [ ]:
print(stats_df)

In [ ]:
plt.figure()
ax = sns.barplot(data=stats_df,
                 x="Tilt series",
                 y="Error weighted mean (nm)",
                 color="cornflowerblue"
                )

ax.errorbar(x=stats_df.index,
            y=stats_df["Error weighted mean (nm)"],
            yerr=stats_df["Error SD (nm)"], fmt="none", c="k")

plt.gca().set_ylim(bottom=0)